In [1]:
import pandas as pd
import json 
from collections import defaultdict
import firebase_admin
from firebase_admin import credentials, auth 
from firebase_admin import db, firestore 
import datetime
import numpy as np

<h1>Firebase
 

In [2]:
cred = credentials.Certificate(r"D:\Workstuff\my-work-python-script\Database_in_pandas\Firebase_Shipment_Database\json_secret\logistec-firebase-adminsdk-mz58b-63b513fce5.json")
firebase_admin.initialize_app(cred, {})

firestore_db = firestore.client()

<h1>UTIL_49

In [15]:
#read clipboard
data = pd.read_clipboard(sep='\t')

#create new dataframe with selected columns
stockout_id = data["ID"].astype(str).str.replace(",","")
branchid = data["Branch (ID)"].astype(str).str.replace(",","")
branchname = data["Branch (Name)"]
phyid = pd.DataFrame(stockout_id + "-" + branchid).rename(columns={0: "PHYID"}, inplace=False)
docdate = pd.to_datetime(data["Doc Date"], dayfirst=True).dt.strftime("%d/%m/%y")
productcode = data["Product (Code)"]
productname = data["Product (Name)"]
totalnumber = data["Number"]
cat_name = data["Category Name"]
serial = data["Serial"].replace({np.nan: "null"})

frames = [docdate,branchname, phyid, productcode, productname, serial, totalnumber, cat_name ]
new_frames = pd.concat(frames, axis=1)

#grouping every columns again to sum all duplicates + set new index.
grouped = new_frames.groupby(['Doc Date', 'Branch (Name)', 'PHYID', 'Product (Code)', 'Product (Name)', 'Serial', 'Category Name'], as_index=False, dropna=False).agg({'Number' : 'sum'}).set_index(["PHYID", 'Product (Code)'], drop=False)
grouped_phyidonly = new_frames["PHYID"].unique()

#convert into dictionary (maps) for firestore
nested_dict = lambda: defaultdict(dict)
nested = nested_dict()

for row2 in grouped.itertuples():
    nested[row2.Index[0]][str(row2.Index[1])] = {'PHYID' : row2.Index[0], 'Doc Date' : row2._1, 'Branch' : row2._2, 'Product Code' : row2.Index[1], 'Product Name' : row2._5, 'Serial' : row2.Serial, 'Number' :row2.Number, 'Category Name' : row2._7}


<H1>SEND DATA

In [ ]:
#send to firestore
print(len(nested))
today = datetime.datetime.today()
batch = firestore_db.batch()

#batch_size = 500
for number, field in enumerate(grouped_phyidonly):
    return49_data = firestore_db.collection("Return49_Data").document("Year").collection(str(today.year)).document("Month").collection(str(today.month - 1)).document(field)
    if number % 500 == 499:
        batch.commit()
        batch = firestore_db.batch()
    batch.set(return49_data, nested[field],merge=True)
batch.commit()